In [50]:


import pandas as pd
import numpy as np
import csv
#Data visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
sns.set(font_scale=1)
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
#Modeling
from sklearn.model_selection import cross_val_predict, cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn_crfsuite import CRF, scorers, metrics
from sklearn.metrics import classification_report, make_scorer
from sklearn.model_selection import GridSearchCV
import scipy.stats
import eli5

In [51]:
data = pd.read_csv("ground_truth4ML.csv",names=["row_num", "word", "tag"],sep='\t',encoding='utf8',quoting=csv.QUOTE_NONE)
onb = pd.read_csv("onb_model_output.csv",names=["word", "lower", "known", "tag" ,"prior_tag"],sep='\t',encoding='utf8' , quotechar = '"')
lft = pd.read_csv("lft_model_output.csv",names=["word", "lower", "known", "tag" ,"prior_tag"],sep='\t',encoding='utf8' , quotechar = '"')

In [52]:
data = data.fillna("O")
#data = data.iloc[1:]
print(len(data))
data.head(20)

44401


,row_num,word,tag
0,665976,(,O
1,665977,10600,O
2,665978,Fuss,O
3,665979,=,O
4,665980,3350,O
5,665981,Meter,O
6,665982,Keil,O
7,665983,.,O
8,665984,),O
9,665985,Sonntag,O


In [16]:
onb.head(50)

,word,lower,known,tag,prior_tag
0,(,(,KNOWN,O,NaN
1,10600,$NUM$,KNOWN,O,NaN
2,Fuss,fuss,KNOWN,O,NaN
3,=,=,KNOWN,O,NaN
4,3350,$NUM$,KNOWN,O,NaN
5,Meter,meter,KNOWN,O,NaN
6,Keil,keil,KNOWN,O,NaN
7,.,.,KNOWN,O,NaN
8,),),KNOWN,O,NaN
9,Sonntag,sonntag,KNOWN,O,NaN


In [17]:
lft.head(50)

,word,lower,known,tag,prior_tag
0,(,(,KNOWN,O,NaN
1,10600,$NUM$,KNOWN,O,NaN
2,Fuss,fuss,KNOWN,O,NaN
3,=,=,KNOWN,O,NaN
4,3350,$NUM$,KNOWN,O,NaN
5,Meter,meter,KNOWN,O,NaN
6,Keil,keil,KNOWN,O,NaN
7,.,.,KNOWN,O,NaN
8,),),KNOWN,O,NaN
9,Sonntag,sonntag,KNOWN,O,NaN


In [53]:
gt =  data.loc[:, ['tag']]
onb_pred = onb.loc[:,['tag']]
lft_pred = lft.loc[:,['tag']]

In [54]:
print(len(gt))
print(len(onb_pred))
print(len(lft_pred))

44401
44401
44401


In [55]:
gt.head(50)

,tag
0,O
1,O
2,O
3,O
4,O
5,O
6,O
7,O
8,O
9,O


In [56]:
report = flat_classification_report(y_pred=lft_pred.values, y_true=gt.values)
print(report)

              precision    recall  f1-score   support

       B-LOC       0.84      0.17      0.29      1980
       B-ORG       0.00      0.00      0.00         0
       B-PER       0.00      0.00      0.00         0
       I-LOC       0.70      0.40      0.51       336
       I-ORG       0.00      0.00      0.00         0
       I-PER       0.00      0.00      0.00         0
           O       0.96      0.99      0.97     42085
         nan       0.00      0.00      0.00         0

   micro avg       0.95      0.95      0.95     44401
   macro avg       0.31      0.19      0.22     44401
weighted avg       0.95      0.95      0.94     44401



In [57]:
report = flat_classification_report(y_pred=onb_pred.values, y_true=gt.values)
print(report)

              precision    recall  f1-score   support

       B-LOC       0.65      0.20      0.30      1980
       B-ORG       0.00      0.00      0.00         0
       B-PER       0.00      0.00      0.00         0
       I-LOC       0.79      0.24      0.37       336
       I-ORG       0.00      0.00      0.00         0
       I-PER       0.00      0.00      0.00         0
           O       0.96      0.99      0.97     42085
         nan       0.00      0.00      0.00         0

   micro avg       0.95      0.95      0.95     44401
   macro avg       0.30      0.18      0.21     44401
weighted avg       0.95      0.95      0.94     44401

